In [ ]:
# import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st

In [ ]:
# import resources
main_df = pd.read_csv("Resources/Cleaned Data.csv")
ad_tag_view_counts_df = pd.read_csv("Resources/ad_tag_view_counts.csv")
ad_tag_counts_df = pd.read_csv("Resources/ad_tag_counts.csv")

# Hypothesis Test

###### Hypothesis
Ads that contain the tag 'Funny' would receive more views on YouTube.

###### Null Hypothesis
There is no statistically significant difference in view counts between ads that contain the 'Funny' tag and those that do not contain the 'Funny' tag.

###### Results
Because the calculated P-Value is exactly zero, we reject the null hypothesis.

In [ ]:
# extract the relevant data from the main DataFrame
funnySeries = main_df.loc[main_df["Funny"] == True, :]["View Counts"]
unfunnySeries = main_df.loc[main_df["Funny"] == False, :]["View Counts"]

In [ ]:
# calculate the expected value
expValue = (sum(funnySeries) + sum(unfunnySeries)) / 2

# construct the hypothesis test dataframe
df0 = pd.DataFrame({
                "observed": [sum(funnySeries), sum(unfunnySeries)],
                "expected": [expValue, expValue]})

In [ ]:
# perform a Chi-Squared test because this is comparing categorical data
pValue = st.chisquare(df0["observed"])[1]

# display the p-value
print(f"P-Value = {pValue:,.10f}")